# Loss Function Comparison: Preventing Middle Value Predictions

This notebook systematically tests different loss functions to prevent models from getting stuck predicting middle values. We'll implement and compare:

1. **Standard Loss Functions**: MSE, MAE, Huber
2. **Custom Loss Functions**: Weighted MSE, Focal Loss for Regression, Quantile Loss
3. **Penalty-based Loss Functions**: Anti-middle penalty, Diversity loss
4. **Ensemble Approaches**: Multiple loss combinations

**Goal**: Find loss functions that encourage prediction diversity while maintaining accuracy.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.losses import Loss
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

In [ ]:
# Load the training data
train_data = pd.read_parquet('../../data/train_balanced.parquet')
validation_data = pd.read_parquet('../../data/validation.parquet')

print(f"Training data shape: {train_data.shape}")
print(f"Validation data shape: {validation_data.shape}")

# Separate features and target
feature_cols = [col for col in train_data.columns if col != 'target']
X_train = train_data[feature_cols].values
y_train = train_data['target'].values
X_val = validation_data[feature_cols].values
y_val = validation_data['target'].values

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

print(f"Feature columns: {len(feature_cols)}")
print(f"Target distribution - Train: {y_train.min():.3f} to {y_train.max():.3f}")
print(f"Target distribution - Val: {y_val.min():.3f} to {y_val.max():.3f}")

In [ ]:
# Analyze target distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Training target distribution
axes[0,0].hist(y_train, bins=50, alpha=0.7, color='blue', edgecolor='black')
axes[0,0].set_title('Training Target Distribution')
axes[0,0].set_xlabel('Target Value')
axes[0,0].set_ylabel('Frequency')
axes[0,0].axvline(y_train.mean(), color='red', linestyle='--', label=f'Mean: {y_train.mean():.3f}')
axes[0,0].legend()

# Validation target distribution
axes[0,1].hist(y_val, bins=50, alpha=0.7, color='green', edgecolor='black')
axes[0,1].set_title('Validation Target Distribution')
axes[0,1].set_xlabel('Target Value')
axes[0,1].set_ylabel('Frequency')
axes[0,1].axvline(y_val.mean(), color='red', linestyle='--', label=f'Mean: {y_val.mean():.3f}')
axes[0,1].legend()

# Box plots
axes[1,0].boxplot([y_train, y_val], labels=['Train', 'Validation'])
axes[1,0].set_title('Target Distribution Comparison')
axes[1,0].set_ylabel('Target Value')

# Percentiles analysis
percentiles = [5, 10, 25, 50, 75, 90, 95]
train_percentiles = [np.percentile(y_train, p) for p in percentiles]
val_percentiles = [np.percentile(y_val, p) for p in percentiles]

axes[1,1].plot(percentiles, train_percentiles, 'o-', label='Train', linewidth=2)
axes[1,1].plot(percentiles, val_percentiles, 's-', label='Validation', linewidth=2)
axes[1,1].set_title('Percentile Analysis')
axes[1,1].set_xlabel('Percentile')
axes[1,1].set_ylabel('Target Value')
axes[1,1].legend()
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nTarget Statistics:")
print(f"Train - Mean: {y_train.mean():.4f}, Std: {y_train.std():.4f}")
print(f"Val - Mean: {y_val.mean():.4f}, Std: {y_val.std():.4f}")
print(f"\nMiddle range (25th-75th percentile):")
print(f"Train: {np.percentile(y_train, 25):.4f} - {np.percentile(y_train, 75):.4f}")
print(f"Val: {np.percentile(y_val, 25):.4f} - {np.percentile(y_val, 75):.4f}")

In [ ]:
# Custom Loss Functions

class WeightedMSELoss(Loss):
    """MSE with higher weights for extreme values to discourage middle predictions"""
    def __init__(self, center=0.5, extreme_weight=2.0, name="weighted_mse"):
        super().__init__(name=name)
        self.center = center
        self.extreme_weight = extreme_weight
    
    def call(self, y_true, y_pred):
        # Calculate distance from center
        distance_from_center = tf.abs(y_true - self.center)
        # Weight: higher for values far from center
        weights = 1.0 + self.extreme_weight * distance_from_center
        # Weighted MSE
        mse = tf.square(y_true - y_pred)
        return tf.reduce_mean(weights * mse)

class FocalRegressionLoss(Loss):
    """Focal loss adapted for regression to focus on hard examples"""
    def __init__(self, alpha=1.0, gamma=2.0, name="focal_regression"):
        super().__init__(name=name)
        self.alpha = alpha
        self.gamma = gamma
    
    def call(self, y_true, y_pred):
        # Calculate absolute error
        abs_error = tf.abs(y_true - y_pred)
        # Focal weight: higher for larger errors
        focal_weight = self.alpha * tf.pow(abs_error, self.gamma)
        # MSE with focal weighting
        mse = tf.square(y_true - y_pred)
        return tf.reduce_mean(focal_weight * mse)

class AntiMiddleLoss(Loss):
    """Loss that penalizes predictions in the middle range"""
    def __init__(self, center=0.5, penalty_range=0.3, penalty_strength=2.0, name="anti_middle"):
        super().__init__(name=name)
        self.center = center
        self.penalty_range = penalty_range
        self.penalty_strength = penalty_strength
    
    def call(self, y_true, y_pred):
        # Standard MSE
        mse = tf.square(y_true - y_pred)
        
        # Penalty for predicting in middle range when true value is not middle
        pred_in_middle = tf.abs(y_pred - self.center) < self.penalty_range
        true_not_middle = tf.abs(y_true - self.center) >= self.penalty_range
        
        # Apply penalty when predicting middle but truth is not middle
        penalty_mask = tf.logical_and(pred_in_middle, true_not_middle)
        penalty = tf.where(penalty_mask, self.penalty_strength, 1.0)
        
        return tf.reduce_mean(penalty * mse)

class QuantileLoss(Loss):
    """Quantile loss to encourage diverse predictions"""
    def __init__(self, quantiles=[0.1, 0.5, 0.9], name="quantile"):
        super().__init__(name=name)
        self.quantiles = quantiles
    
    def call(self, y_true, y_pred):
        # For simplicity, use asymmetric loss encouraging spread
        errors = y_true - y_pred
        # Asymmetric penalty
        loss = tf.where(errors >= 0, 0.7 * tf.abs(errors), 1.3 * tf.abs(errors))
        return tf.reduce_mean(loss)

class DiversityLoss(Loss):
    """Loss that encourages prediction diversity within batches"""
    def __init__(self, diversity_weight=0.1, name="diversity"):
        super().__init__(name=name)
        self.diversity_weight = diversity_weight
    
    def call(self, y_true, y_pred):
        # Standard MSE
        mse = tf.reduce_mean(tf.square(y_true - y_pred))
        
        # Diversity penalty: encourage variance in predictions
        pred_var = tf.reduce_mean(tf.square(y_pred - tf.reduce_mean(y_pred)))
        diversity_bonus = -self.diversity_weight * pred_var  # Negative to encourage diversity
        
        return mse + diversity_bonus

# Standard loss functions for comparison
def get_loss_functions():
    return {
        'mse': 'mse',
        'mae': 'mae', 
        'huber': tf.keras.losses.Huber(delta=0.1),
        'weighted_mse': WeightedMSELoss(center=0.5, extreme_weight=2.0),
        'focal_regression': FocalRegressionLoss(alpha=1.0, gamma=2.0),
        'anti_middle': AntiMiddleLoss(center=0.5, penalty_range=0.3, penalty_strength=2.0),
        'quantile': QuantileLoss(),
        'diversity': DiversityLoss(diversity_weight=0.1)
    }

print("Custom loss functions defined:")
for name in get_loss_functions().keys():
    print(f"- {name}")

In [ ]:
def create_model(input_dim, loss_function, learning_rate=0.001):
    """Create a neural network model with specified loss function"""
    model = models.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # Output in [0,1] range
    ])
    
    optimizer = optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=loss_function, metrics=['mae'])
    
    return model

def train_and_evaluate_model(loss_name, loss_function, X_train, y_train, X_val, y_val, epochs=50, verbose=0):
    """Train model with specific loss function and return results"""
    print(f"\nTraining model with {loss_name} loss...")
    
    # Create model
    model = create_model(X_train.shape[1], loss_function)
    
    # Callbacks
    early_stopping = callbacks.EarlyStopping(
        monitor='val_loss', patience=10, restore_best_weights=True
    )
    
    reduce_lr = callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6
    )
    
    # Train model
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=32,
        callbacks=[early_stopping, reduce_lr],
        verbose=verbose
    )
    
    # Make predictions
    train_pred = model.predict(X_train, verbose=0).flatten()
    val_pred = model.predict(X_val, verbose=0).flatten()
    
    # Calculate metrics
    train_mse = mean_squared_error(y_train, train_pred)
    val_mse = mean_squared_error(y_val, val_pred)
    train_mae = mean_absolute_error(y_train, train_pred)
    val_mae = mean_absolute_error(y_val, val_pred)
    train_r2 = r2_score(y_train, train_pred)
    val_r2 = r2_score(y_val, val_pred)
    
    # Calculate prediction diversity metrics
    train_pred_std = np.std(train_pred)
    val_pred_std = np.std(val_pred)
    train_pred_range = np.max(train_pred) - np.min(train_pred)
    val_pred_range = np.max(val_pred) - np.min(val_pred)
    
    # Calculate middle clustering metric (percentage of predictions in middle 50%)
    middle_lower = np.percentile(y_train, 25)
    middle_upper = np.percentile(y_train, 75)
    train_middle_pct = np.mean((train_pred >= middle_lower) & (train_pred <= middle_upper)) * 100
    val_middle_pct = np.mean((val_pred >= middle_lower) & (val_pred <= middle_upper)) * 100
    
    results = {
        'loss_name': loss_name,
        'model': model,
        'history': history,
        'train_pred': train_pred,
        'val_pred': val_pred,
        'train_mse': train_mse,
        'val_mse': val_mse,
        'train_mae': train_mae,
        'val_mae': val_mae,
        'train_r2': train_r2,
        'val_r2': val_r2,
        'train_pred_std': train_pred_std,
        'val_pred_std': val_pred_std,
        'train_pred_range': train_pred_range,
        'val_pred_range': val_pred_range,
        'train_middle_pct': train_middle_pct,
        'val_middle_pct': val_middle_pct
    }
    
    print(f"Validation MSE: {val_mse:.4f}, Prediction Std: {val_pred_std:.4f}, Middle %: {val_middle_pct:.1f}%")
    
    return results

print("Model training functions defined.")

In [ ]:
# Train models with different loss functions
loss_functions = get_loss_functions()
results = {}

print("Starting training with different loss functions...")
print("=" * 60)

for loss_name, loss_function in loss_functions.items():
    try:
        result = train_and_evaluate_model(
            loss_name, loss_function, 
            X_train_scaled, y_train, 
            X_val_scaled, y_val,
            epochs=100, verbose=0
        )
        results[loss_name] = result
    except Exception as e:
        print(f"Error training {loss_name}: {str(e)}")
        continue

print("\n" + "=" * 60)
print("Training completed!")

In [ ]:
# Create results comparison table
comparison_data = []

for loss_name, result in results.items():
    comparison_data.append({
        'Loss Function': loss_name,
        'Val MSE': result['val_mse'],
        'Val MAE': result['val_mae'],
        'Val R²': result['val_r2'],
        'Pred Std': result['val_pred_std'],
        'Pred Range': result['val_pred_range'],
        'Middle %': result['val_middle_pct']
    })

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('Val MSE')

print("\n" + "=" * 80)
print("LOSS FUNCTION COMPARISON RESULTS")
print("=" * 80)
print("\nMetrics explanation:")
print("- Val MSE: Validation Mean Squared Error (lower is better)")
print("- Val R²: R-squared score (higher is better)")
print("- Pred Std: Standard deviation of predictions (higher = more diverse)")
print("- Pred Range: Range of predictions (higher = more diverse)")
print("- Middle %: Percentage of predictions in middle 50% range (lower = less clustering)")
print("\n")

# Display results
for idx, row in comparison_df.iterrows():
    print(f"{row['Loss Function']:15} | MSE: {row['Val MSE']:.4f} | R²: {row['Val R²']:.3f} | "
          f"Std: {row['Pred Std']:.3f} | Range: {row['Pred Range']:.3f} | Middle: {row['Middle %']:.1f}%")

print("\n" + "=" * 80)

In [ ]:
# Visualize prediction distributions
num_losses = len(results)
fig, axes = plt.subplots(3, 3, figsize=(18, 15))
axes = axes.flatten()

for idx, (loss_name, result) in enumerate(results.items()):
    if idx >= len(axes):
        break
        
    ax = axes[idx]
    
    # Plot prediction distribution
    ax.hist(result['val_pred'], bins=30, alpha=0.7, color='skyblue', 
            edgecolor='black', label='Predictions')
    ax.hist(y_val, bins=30, alpha=0.5, color='orange', 
            edgecolor='black', label='True Values')
    
    ax.set_title(f'{loss_name}\nMSE: {result["val_mse"]:.4f}, Std: {result["val_pred_std"]:.3f}')
    ax.set_xlabel('Value')
    ax.set_ylabel('Frequency')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Add statistics text
    stats_text = f"Range: {result['val_pred_range']:.3f}\nMiddle %: {result['val_middle_pct']:.1f}%"
    ax.text(0.02, 0.98, stats_text, transform=ax.transAxes, 
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# Hide empty subplots
for idx in range(len(results), len(axes)):
    axes[idx].set_visible(False)

plt.tight_layout()
plt.suptitle('Prediction Distributions by Loss Function', fontsize=16, y=1.02)
plt.show()

In [ ]:
# Scatter plots: True vs Predicted values
fig, axes = plt.subplots(3, 3, figsize=(18, 15))
axes = axes.flatten()

for idx, (loss_name, result) in enumerate(results.items()):
    if idx >= len(axes):
        break
        
    ax = axes[idx]
    
    # Create scatter plot
    ax.scatter(y_val, result['val_pred'], alpha=0.6, s=20)
    
    # Add perfect prediction line
    min_val = min(y_val.min(), result['val_pred'].min())
    max_val = max(y_val.max(), result['val_pred'].max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')
    
    # Add horizontal line at middle value
    middle_val = 0.5
    ax.axhline(y=middle_val, color='orange', linestyle=':', alpha=0.7, label='Middle Value')
    
    ax.set_title(f'{loss_name}\nR²: {result["val_r2"]:.3f}')
    ax.set_xlabel('True Values')
    ax.set_ylabel('Predicted Values')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Set equal aspect ratio
    ax.set_aspect('equal', adjustable='box')

# Hide empty subplots
for idx in range(len(results), len(axes)):
    axes[idx].set_visible(False)

plt.tight_layout()
plt.suptitle('True vs Predicted Values by Loss Function', fontsize=16, y=1.02)
plt.show()

In [ ]:
# Detailed analysis of anti-middle clustering performance
print("\n" + "=" * 80)
print("ANTI-MIDDLE CLUSTERING ANALYSIS")
print("=" * 80)

# Sort by different criteria
print("\n1. LOWEST MIDDLE CLUSTERING (Best for avoiding middle predictions):")
print("-" * 60)
anti_middle_ranking = comparison_df.sort_values('Middle %')
for idx, row in anti_middle_ranking.head().iterrows():
    print(f"{row['Loss Function']:15} | Middle %: {row['Middle %']:5.1f}% | "
          f"Pred Std: {row['Pred Std']:.3f} | Val MSE: {row['Val MSE']:.4f}")

print("\n2. HIGHEST PREDICTION DIVERSITY (Best for varied predictions):")
print("-" * 60)
diversity_ranking = comparison_df.sort_values('Pred Std', ascending=False)
for idx, row in diversity_ranking.head().iterrows():
    print(f"{row['Loss Function']:15} | Pred Std: {row['Pred Std']:.3f} | "
          f"Range: {row['Pred Range']:.3f} | Val MSE: {row['Val MSE']:.4f}")

print("\n3. BEST BALANCE (Low MSE + Low Middle Clustering):")
print("-" * 60)
# Create composite score: normalize metrics and combine
comparison_df['MSE_norm'] = (comparison_df['Val MSE'] - comparison_df['Val MSE'].min()) / (comparison_df['Val MSE'].max() - comparison_df['Val MSE'].min())
comparison_df['Middle_norm'] = (comparison_df['Middle %'] - comparison_df['Middle %'].min()) / (comparison_df['Middle %'].max() - comparison_df['Middle %'].min())
comparison_df['Composite_Score'] = comparison_df['MSE_norm'] + comparison_df['Middle_norm']  # Lower is better

balanced_ranking = comparison_df.sort_values('Composite_Score')
for idx, row in balanced_ranking.head().iterrows():
    print(f"{row['Loss Function']:15} | Composite: {row['Composite_Score']:.3f} | "
          f"MSE: {row['Val MSE']:.4f} | Middle %: {row['Middle %']:5.1f}%")

print("\n" + "=" * 80)

In [ ]:
# Summary visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. MSE vs Middle Clustering
axes[0,0].scatter(comparison_df['Middle %'], comparison_df['Val MSE'], s=100, alpha=0.7)
for idx, row in comparison_df.iterrows():
    axes[0,0].annotate(row['Loss Function'], 
                      (row['Middle %'], row['Val MSE']),
                      xytext=(5, 5), textcoords='offset points', fontsize=9)
axes[0,0].set_xlabel('Middle Clustering %')
axes[0,0].set_ylabel('Validation MSE')
axes[0,0].set_title('MSE vs Middle Clustering\n(Lower left is better)')
axes[0,0].grid(True, alpha=0.3)

# 2. Prediction Diversity vs Accuracy
axes[0,1].scatter(comparison_df['Pred Std'], comparison_df['Val R²'], s=100, alpha=0.7)
for idx, row in comparison_df.iterrows():
    axes[0,1].annotate(row['Loss Function'], 
                      (row['Pred Std'], row['Val R²']),
                      xytext=(5, 5), textcoords='offset points', fontsize=9)
axes[0,1].set_xlabel('Prediction Standard Deviation')
axes[0,1].set_ylabel('Validation R²')
axes[0,1].set_title('Prediction Diversity vs Accuracy\n(Upper right is better)')
axes[0,1].grid(True, alpha=0.3)

# 3. Ranking comparison
loss_names = comparison_df['Loss Function'].tolist()
y_pos = np.arange(len(loss_names))

axes[1,0].barh(y_pos, comparison_df['Middle %'], alpha=0.7, color='coral')
axes[1,0].set_yticks(y_pos)
axes[1,0].set_yticklabels(loss_names)
axes[1,0].set_xlabel('Middle Clustering %')
axes[1,0].set_title('Middle Clustering by Loss Function\n(Lower is better)')
axes[1,0].grid(True, alpha=0.3, axis='x')

# 4. Composite score ranking
balanced_ranking_display = balanced_ranking.copy()
axes[1,1].barh(y_pos, balanced_ranking_display['Composite_Score'], alpha=0.7, color='lightblue')
axes[1,1].set_yticks(y_pos)
axes[1,1].set_yticklabels(balanced_ranking_display['Loss Function'])
axes[1,1].set_xlabel('Composite Score (MSE + Middle %)')
axes[1,1].set_title('Overall Performance Ranking\n(Lower is better)')
axes[1,1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

In [ ]:
# Generate recommendations
print("\n" + "=" * 80)
print("RECOMMENDATIONS TO PREVENT MIDDLE VALUE CLUSTERING")
print("=" * 80)

# Get best performers in each category
best_anti_middle = anti_middle_ranking.iloc[0]
best_diversity = diversity_ranking.iloc[0]
best_balanced = balanced_ranking.iloc[0]

print("\n🎯 TOP RECOMMENDATIONS:")
print("-" * 40)
print(f"\n1. BEST ANTI-MIDDLE CLUSTERING: {best_anti_middle['Loss Function']}")
print(f"   - Middle clustering: {best_anti_middle['Middle %']:.1f}%")
print(f"   - Validation MSE: {best_anti_middle['Val MSE']:.4f}")
print(f"   - Prediction diversity: {best_anti_middle['Pred Std']:.3f}")

print(f"\n2. HIGHEST PREDICTION DIVERSITY: {best_diversity['Loss Function']}")
print(f"   - Prediction std: {best_diversity['Pred Std']:.3f}")
print(f"   - Prediction range: {best_diversity['Pred Range']:.3f}")
print(f"   - Validation MSE: {best_diversity['Val MSE']:.4f}")

print(f"\n3. BEST OVERALL BALANCE: {best_balanced['Loss Function']}")
print(f"   - Composite score: {best_balanced['Composite_Score']:.3f}")
print(f"   - Validation MSE: {best_balanced['Val MSE']:.4f}")
print(f"   - Middle clustering: {best_balanced['Middle %']:.1f}%")

print("\n📋 IMPLEMENTATION STRATEGIES:")
print("-" * 40)
print("\n• Use weighted MSE with higher weights for extreme values")
print("• Implement anti-middle penalty in loss function")
print("• Add diversity regularization to encourage prediction spread")
print("• Consider focal loss for focusing on hard examples")
print("• Experiment with quantile loss for asymmetric penalties")

print("\n⚠️  KEY INSIGHTS:")
print("-" * 40)
print("• Standard MSE tends to produce middle-clustered predictions")
print("• Custom loss functions can significantly reduce middle clustering")
print("• There's often a trade-off between accuracy and prediction diversity")
print("• Weighted and penalty-based losses show most promise")

print("\n🚀 NEXT STEPS:")
print("-" * 40)
print("• Fine-tune the best-performing loss function hyperparameters")
print("• Test ensemble methods combining multiple loss functions")
print("• Implement data augmentation techniques")
print("• Consider architectural changes (e.g., multiple output heads)")

print("\n" + "=" * 80)

In [ ]:
# Save results and best models
print("\nSaving results and best models...")

# Save comparison results to CSV
comparison_df.to_csv('../../data/loss_function_comparison.csv', index=False)
print("✅ Comparison results saved to: data/loss_function_comparison.csv")

# Save the best model from each category
best_models = {
    'anti_middle': best_anti_middle['Loss Function'],
    'diversity': best_diversity['Loss Function'], 
    'balanced': best_balanced['Loss Function']
}

for category, loss_name in best_models.items():
    if loss_name in results:
        model_path = f'../../exports/best_{category}_{loss_name}.keras'
        results[loss_name]['model'].save(model_path)
        print(f"✅ Best {category} model ({loss_name}) saved to: {model_path}")

print("\n📊 FINAL SUMMARY:")
print("-" * 40)
print(f"• Tested {len(results)} different loss functions")
print(f"• Best anti-middle: {best_anti_middle['Loss Function']} ({best_anti_middle['Middle %']:.1f}% clustering)")
print(f"• Most diverse: {best_diversity['Loss Function']} (std: {best_diversity['Pred Std']:.3f})")
print(f"• Best balanced: {best_balanced['Loss Function']} (score: {best_balanced['Composite_Score']:.3f})")
print("\n🎉 Loss function analysis completed successfully!")
print("\nUse the saved models and insights to improve your prediction diversity.")